In [1]:
import pandas as pd
import statsmodels.api as sm

In [2]:
df = pd.read_csv("../data/processed/macro_monthly_clean.csv")

df = df.sort_values(["year", "month"]).reset_index(drop=True)
df.head()

,year,month,bi_rate,inflation,period
0,2016,4,5.50,3.60,2016-04
1,2016,5,5.50,3.33,2016-05
2,2016,6,5.25,3.45,2016-06
3,2016,7,5.25,3.21,2016-07
4,2016,8,5.25,2.79,2016-08


In [3]:
def run_lag_regression(df, lag):
    data = df.copy()
    
    # lag BI Rate
    data["bi_rate_lag"] = data["bi_rate"].shift(lag)
    
    # drop missing akibat lag
    data = data.dropna()
    
    X = sm.add_constant(data["bi_rate_lag"])
    y = data["inflation"]
    
    model = sm.OLS(y, X).fit()
    
    return model

In [4]:
models = {}

for lag in range(1, 5):
    model = run_lag_regression(df, lag)
    models[lag] = model
    
    print(f"\n=== Regression Result (Lag {lag}) ===")
    print(model.summary())


=== Regression Result (Lag 1) ===
                            OLS Regression Results                            
Dep. Variable:              inflation   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.3669
Date:                Mon, 09 Feb 2026   Prob (F-statistic):              0.546
Time:                        16:38:07   Log-Likelihood:                -177.29
No. Observations:                 117   AIC:                             358.6
Df Residuals:                     115   BIC:                             364.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          

In [5]:
results = []

for lag, model in models.items():
    results.append({
        "lag_month": lag,
        "coef_bi_rate": model.params["bi_rate_lag"],
        "p_value": model.pvalues["bi_rate_lag"],
        "r_squared": model.rsquared
    })

df_regression_results = pd.DataFrame(results)

df_regression_results

,lag_month,coef_bi_rate,p_value,r_squared
0,1,0.068986,0.545910,0.003180
1,2,-0.020763,0.856428,0.000288
2,3,-0.108197,0.346528,0.007846
3,4,-0.186220,0.105318,0.023251


In [6]:
df_regression_results.to_csv(
    "../data/processed/regression_lag_coefficients.csv",
    index=False
)

In [7]:
with open("../data/processed/regression_model_summaries.txt", "w") as f:
    for lag, model in models.items():
        f.write(f"\n=== Regression Result (Lag {lag}) ===\n")
        f.write(model.summary().as_text())
        f.write("\n\n")

### Step 5 — Regression Analysis with Lagged BI Rate

This step extends the lag correlation analysis by estimating a series of simple linear regression models, 
where inflation is regressed on lagged values of BI Rate. The goal is to assess whether the lagged relationship 
remains consistent when evaluated in a regression framework.

### Model Specification

For each lag \( k \), the following regression model is estimated:

$$
\text{Inflation}_t = \alpha + \beta \cdot \text{BI Rate}_{t-k} + \varepsilon_t
$$

where \( k \) ranges from 1 to 4 months.

### Key Findings

- At lag 1 and lag 2, the estimated coefficients are small and statistically insignificant, indicating no immediate relationship.
- At lag 3, the coefficient becomes negative, although it remains statistically insignificant.
- At lag 4, the coefficient is more negative and relatively larger in magnitude, with a p-value close to conventional significance thresholds.

### Interpretation

The regression results suggest that the relationship between BI Rate and inflation is not instantaneous. 
Instead, the negative association becomes more pronounced at longer lags, particularly around four months. 
While the estimated effects are modest and not strongly significant, the consistent change in coefficient signs 
across lags supports the presence of a delayed response of inflation to monetary policy changes.

Overall, these results reinforce the findings from the exploratory and correlation analyses, indicating that 
any potential impact of BI Rate on inflation operates with a time lag rather than contemporaneously.